<a href="https://colab.research.google.com/github/YuxinZhang1998/S3-SIF/blob/main/SIF_Output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
import geemap
import random
import datetime
import os
import numpy as np
import pandas as pd
from scipy import signal

ee.Authenticate()
ee.Initialize(project='ee-yuxinzhanguv1')

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


In [ ]:
TROPOMI = ee.ImageCollection('projects/ee-yuxinzhanguv1/assets/TROPOMI2019')

def add_timestamp(image):
    id_no = ee.String(image.get('id_no'))  # e.g., "TROPOMISIF_8days_20190701"
    date_str = id_no.slice(-8)             # Extract "20190701"
    date = ee.Date.parse('yyyyMMdd', date_str)
    return image.set('system:time_start', date.millis())

# Apply the function to the collection
collection_with_time = TROPOMI.map(add_timestamp)
SIF = collection_with_time.select('b1')

S3 = ee.ImageCollection('projects/ee-yuxinzhanguv1/assets/S3-4DAYS-2019-FeatureCollection')
FVC = ee.ImageCollection('projects/ee-yuxinzhanguv1/assets/GPR-S3-4DAYS-FVC-2019').select(['FVC_GREEN']);
LAI = ee.ImageCollection('projects/ee-yuxinzhanguv1/assets/GPR-S3-4DAYS-LAI-2019').select(['LAI_GREEN']);
LCC = ee.ImageCollection('projects/ee-yuxinzhanguv1/assets/GPR-S3-4DAYS-LCC-2019').select(['LCC_GREEN']);
Geo = ee.ImageCollection('projects/ee-yuxinzhanguv1/assets/Geo-4DAYS-2019-FeatureCollection').select(['lon','lat']);

region = ee.Geometry.Polygon(
    [[[-28.0, 81.5], [-28.0, 25.5], [67, 25.5], [67, 81.5], [-28.0, 81.5]]]
)

def smooth(image):
    boxcar = ee.Kernel.square(radius=3, units='pixels', normalize=True)
    return image.convolve(boxcar).copyProperties(image, image.propertyNames())

smoothed = lambda col: col.map(smooth).filterBounds(region)
smoothedS3 = smoothed(S3)
smoothedFVC = smoothed(FVC)
smoothedLAI = smoothed(LAI)
smoothedLCC = smoothed(LCC)

timestamps = smoothedS3.aggregate_array('system:time_start').getInfo()

def make_training_image(t):
    date = ee.Date(t)
    s3 = smoothedS3.filterDate(date, date.advance(1, 'day')).first()
    fvc = smoothedFVC.filterDate(date, date.advance(1, 'day')).first()
    lai = smoothedLAI.filterDate(date, date.advance(1, 'day')).first()
    lcc = smoothedLCC.filterDate(date, date.advance(1, 'day')).first()
    geo = Geo.filterDate(date, date.advance(1, 'day')).first()
    sif = SIF.filterDate(date, date.advance(1, 'day')).first().rename('SIF')
    return s3.addBands(fvc).addBands(lcc).addBands(geo).addBands(sif)

training_images = ee.ImageCollection([make_training_image(t) for t in timestamps])
startDateStr = '2019-06-01'
endDateStr = '2019-11-30'

training_imagesMid = training_images.filterDate(startDateStr, endDateStr)
samples = training_imagesMid.map(lambda img: img.sample(
    region=region,
    scale=5600,
    numPixels=6000,
    seed=41,
    geometries=True
)).flatten()

samples = samples.randomColumn(columnName='random')
train = samples.filter(ee.Filter.lt('random', 0.8))
test = samples.filter(ee.Filter.gte('random', 0.8))
band_names = training_images.first().bandNames().remove('SIF')

rf_model = ee.Classifier.smileRandomForest(50).setOutputMode('REGRESSION').train(
    features=train,
    classProperty='SIF',
    inputProperties=band_names
)


In [ ]:
pixelsize = 300
startDateStr = '2019-01-01'
endDateStr = '2019-12-31'
S3site = ee.ImageCollection('projects/ee-yuxinzhanguv1/assets/S3-4DAYS-global2019-300-FeatureCollection').filterDate(startDateStr, endDateStr)
FVCsites = ee.ImageCollection('projects/ee-yuxinzhanguv1/assets/GPR-S3-8DAYS-peninsula2019-300-FVC').select(['FVC']).filterDate(startDateStr, endDateStr)
LCCsites = ee.ImageCollection('projects/ee-yuxinzhanguv1/assets/GPR-S3-8DAYS-peninsula2019-300-LCC').select(['LCC']).filterDate(startDateStr, endDateStr)
Geosites = ee.ImageCollection('projects/ee-yuxinzhanguv1/assets/Geo-8DAYS-peninsula2019-300-FeatureCollection').select(['lon','lat']).filterDate(startDateStr, endDateStr)

export_prefix = 'Predicted_peninsula2019_'
asset_prefix = 'projects/ee-yuxinzhanguv1/assets/Predicted_peninsula2019_Collection/'
region = ee.Geometry.Rectangle([-10, 36, 4, 44], geodesic=False)

In [ ]:
# @title In meters
def rename_bands(image):
    return image.rename(new_band_names)

# new_band_names = ['lon', 'lat']
# Geosite = Geosites.map(rename_bands)
new_band_names = ['FVC_GREEN']
FVCsite = FVCsites.map(rename_bands)
new_band_names = ['LCC_GREEN']
LCCsite = LCCsites.map(rename_bands)

image_S3 = S3site.first()
ref_proj = image_S3.projection()
ref_crs = ref_proj.crs()
ref_transform = ref_proj.getInfo()['transform']  # [xscale, 0, xOrigin, 0, yscale, yOrigin]

# def align_image_to_reference(image):
#     return (
#         image
#         .resample('bilinear')
#         .reproject(crs=ref_crs, crsTransform=ref_transform)
#         .clip(image_S3.geometry())
#     )


timestampssite = S3site.aggregate_array('system:time_start').getInfo()

def make_training_site(t):
    date = ee.Date(t)
    s3 = S3site.filterDate(date, date.advance(1, 'day')).first()
    fvc = FVCsite.filterDate(date, date.advance(1, 'day')).first()
    lcc = LCCsite.filterDate(date, date.advance(1, 'day')).first()
    geo = Geosites.filterDate(date, date.advance(1, 'day')).first()
    return s3.addBands(fvc).addBands(lcc).addBands(geo)

inputsite = ee.ImageCollection([make_training_site(t) for t in timestampssite])
input_resampled = inputsite#.map(align_image_to_reference)

def classify_image(image):
  return image.classify(rf_model)

predicted_site = input_resampled.filterDate(startDateStr,endDateStr).map(classify_image)

n_images = predicted_site.size()
image_list = predicted_site.toList(n_images)

# 将索引拉到客户端（很轻量）
for i in range(n_images.getInfo()):
    try:
        # 获取图像并转换为 float
        image = ee.Image(image_list.get(i)).toFloat()

        # 获取 system:index（示例：S3_export_20181023）
        index_str = ee.String(image.get('system:index'))

        # 提取最后8位作为日期字符串：20181023
        date_str = index_str.slice(-8)

        # 生成导出名称和路径
        export_name = export_prefix + date_str.getInfo()
        asset_path = asset_prefix + export_name

        print(f"Processing: {export_name}")

        # 设置日期属性并导出
        image = image.set('date', date_str)

        task = ee.batch.Export.image.toAsset(
            image=image,
            description=export_name,
            region=region,
            assetId=asset_path,
            scale=pixelsize,
            maxPixels=1e13
        )
        task.start()
        print(f"Export task created for {export_name}")

    except Exception as e:
        print(f"Error processing image at index {i}: {e}")
        continue

Processing: Predicted_peninsula2019_20190101
Export task created for Predicted_peninsula2019_20190101
Processing: Predicted_peninsula2019_20190109
Export task created for Predicted_peninsula2019_20190109
Processing: Predicted_peninsula2019_20190117
Export task created for Predicted_peninsula2019_20190117
Processing: Predicted_peninsula2019_20190125
Export task created for Predicted_peninsula2019_20190125
Processing: Predicted_peninsula2019_20190202
Export task created for Predicted_peninsula2019_20190202
Processing: Predicted_peninsula2019_20190210
Export task created for Predicted_peninsula2019_20190210
Processing: Predicted_peninsula2019_20190218
Export task created for Predicted_peninsula2019_20190218
Processing: Predicted_peninsula2019_20190226
Export task created for Predicted_peninsula2019_20190226
Processing: Predicted_peninsula2019_20190306
Export task created for Predicted_peninsula2019_20190306
Processing: Predicted_peninsula2019_20190314
Export task created for Predicted_pen

Processing: Predicted_peninsula2019_20190517
Export task created for Predicted_peninsula2019_20190517
Processing: Predicted_peninsula2019_20190525
Export task created for Predicted_peninsula2019_20190525
Processing: Predicted_peninsula2019_20190602
Export task created for Predicted_peninsula2019_20190602


Processing: Predicted_peninsula2019_20190610
Export task created for Predicted_peninsula2019_20190610
Processing: Predicted_peninsula2019_20190618
Export task created for Predicted_peninsula2019_20190618
Processing: Predicted_peninsula2019_20190626
Export task created for Predicted_peninsula2019_20190626
Processing: Predicted_peninsula2019_20190704
Export task created for Predicted_peninsula2019_20190704
Processing: Predicted_peninsula2019_20190712
Export task created for Predicted_peninsula2019_20190712


Processing: Predicted_peninsula2019_20190720
Export task created for Predicted_peninsula2019_20190720
Processing: Predicted_peninsula2019_20190728
Export task created for Predicted_peninsula2019_20190728
Processing: Predicted_peninsula2019_20190805
Export task created for Predicted_peninsula2019_20190805
Processing: Predicted_peninsula2019_20190813
Export task created for Predicted_peninsula2019_20190813
Processing: Predicted_peninsula2019_20190821
Export task created for Predicted_peninsula2019_20190821
Processing: Predicted_peninsula2019_20190829
Export task created for Predicted_peninsula2019_20190829
Processing: Predicted_peninsula2019_20190906
Export task created for Predicted_peninsula2019_20190906


Processing: Predicted_peninsula2019_20190914
Export task created for Predicted_peninsula2019_20190914
Processing: Predicted_peninsula2019_20190922
Export task created for Predicted_peninsula2019_20190922
Processing: Predicted_peninsula2019_20190930
Export task created for Predicted_peninsula2019_20190930
Processing: Predicted_peninsula2019_20191008
Export task created for Predicted_peninsula2019_20191008
Processing: Predicted_peninsula2019_20191016
Export task created for Predicted_peninsula2019_20191016
Processing: Predicted_peninsula2019_20191024
Export task created for Predicted_peninsula2019_20191024
Processing: Predicted_peninsula2019_20191101
Export task created for Predicted_peninsula2019_20191101
Processing: Predicted_peninsula2019_20191109
Export task created for Predicted_peninsula2019_20191109
Processing: Predicted_peninsula2019_20191117
Export task created for Predicted_peninsula2019_20191117
Processing: Predicted_peninsula2019_20191125
Export task created for Predicted_pen

In [ ]:
# @title In degree
def rename_bands(image):
    return image.rename(new_band_names)

# new_band_names = ['lon', 'lat']
# Geosite = Geosites.map(rename_bands)
new_band_names = ['FVC_GREEN']
FVCsite = FVCsites.map(rename_bands)
new_band_names = ['LCC_GREEN']
LCCsite = LCCsites.map(rename_bands)

image_S3 = S3site.first()
ref_proj = image_S3.projection()
ref_crs = ref_proj.crs()
ref_transform = ref_proj.getInfo()['transform']  # [xscale, 0, xOrigin, 0, yscale, yOrigin]

# def align_image_to_reference(image):
#     return (
#         image
#         .resample('bilinear')
#         .reproject(crs=ref_crs, crsTransform=ref_transform)
#         .clip(image_S3.geometry())
#     )


timestampssite = S3site.aggregate_array('system:time_start').getInfo()

# 构建每个时间点的 image（包括所有特征 + SIF）
def make_training_site(t):
    date = ee.Date(t)
    s3 = S3site.filterDate(date, date.advance(1, 'day')).first()
    fvc = FVCsite.filterDate(date, date.advance(1, 'day')).first()
    lcc = LCCsite.filterDate(date, date.advance(1, 'day')).first()
    geo = Geosites.filterDate(date, date.advance(1, 'day')).first()
    return s3.addBands(fvc).addBands(lcc).addBands(geo)

inputsite = ee.ImageCollection([make_training_site(t) for t in timestampssite])
input_resampled = inputsite#.map(align_image_to_reference)

def classify_image(image):
  return image.classify(rf_model)

predicted_site = input_resampled.filterDate(startDateStr,endDateStr).map(classify_image)

# 0.05° 投影（度制网格）
proj_05deg = ee.Projection('EPSG:4326').atScale(0.05)

# 定义 crsTransform: [xScale, xShearing, xOrigin, yShearing, yScale, yOrigin]
# 原点左上角 (-180, 90)，像元大小 0.05°
crsTransform_05deg = [0.05, 0, -180, 0, -0.05, 90]

n_images = predicted_site.size()
image_list = predicted_site.toList(n_images)

region = ee.Geometry.Rectangle([-180, -90, 180, 90], geodesic=False)

for i in range(n_images.getInfo()):
    try:
        # 获取图像并转换为 float
        image = ee.Image(image_list.get(i)).toFloat()

        # 获取 system:index（示例：S3_export_20181023）
        index_str = ee.String(image.get('system:index'))

        # 提取最后8位作为日期字符串：20181023
        date_str = index_str.slice(-8)

        # 生成导出名称和路径
        export_name = export_prefix + date_str.getInfo()
        asset_path = asset_prefix + export_name

        print(f"Processing: {export_name}")

        # 设置日期属性
        image = image.set('date', date_str)


        task = ee.batch.Export.image.toDrive(
          image=image,
          description=export_name,
          folder="GEE_exports_SIF2020",    # Drive 文件夹
          #fileNamePrefix=file_name,
          region=region,
          crs="EPSG:4326",
          crsTransform=crsTransform_05deg,
          maxPixels=1e13
    )
        task.start()
        print(f"Export task created for {export_name}")

    except Exception as e:
        print(f"Error processing image at index {i}: {e}")
        continue


Processing: Predicted_global2020_20201003
Export task created for Predicted_global2020_20201003
Processing: Predicted_global2020_20201007
Export task created for Predicted_global2020_20201007
Processing: Predicted_global2020_20201011
Export task created for Predicted_global2020_20201011
Processing: Predicted_global2020_20201015
Export task created for Predicted_global2020_20201015
Processing: Predicted_global2020_20201019
Export task created for Predicted_global2020_20201019
Processing: Predicted_global2020_20201023
Export task created for Predicted_global2020_20201023
Processing: Predicted_global2020_20201027
Export task created for Predicted_global2020_20201027
Processing: Predicted_global2020_20201031
Export task created for Predicted_global2020_20201031
Processing: Predicted_global2020_20201104
Export task created for Predicted_global2020_20201104
Processing: Predicted_global2020_20201108
Export task created for Predicted_global2020_20201108
Processing: Predicted_global2020_2020111